In [32]:
############### Set working directory to parent directory
import os,  requests, sys, re, urllib.request, pandas as pd
from bs4 import BeautifulSoup

os.getcwd()
# os.chdir('F:\\github\\narrative_conservatism\\code')

'F:\\github\\narrative_conservatism\\code'

In [197]:
# Access initial search page ############### loop over all companies?##########################
cik = '0000320193'
type = '8-K'
# number of documents listed per page
count = '100'
# index of first document listed in the current page
start = '0'
# find filings prior to the date 2016y01m01d
dateb = ''

# Obtain url for intial search result page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}&start={}&count={}"
init_url = base_url.format(cik, type, dateb, start, count)

# define a function that takes the input url and returns next search page url
def get_next_url(input_url):
    edgar_resp = requests.get(input_url)
    edgar_str = edgar_resp.text
    soup = BeautifulSoup(edgar_str, 'html.parser')

    div_tag = soup.find('div', style='margin-top: 5px; margin-bottom: 5px;')
    button = div_tag.find('td', style='text-align: right;')
    fbutton = button.find_all('input')[0]['value']
    if re.findall(r'Next', fbutton) == ['Next']:
        next_url = button.find_all('input')[0]['onclick'][:-1]
    elif len(button.find_all('input')) == 2:
        next_url = button.find_all('input')[1]['onclick'][:-1]
    else:
        next_url = 'NA'
        
    next_url = next_url.replace('parent.location=\'', 'https://www.sec.gov')
    return next_url

# create a search result page url list
search_url = [init_url]

while get_next_url(init_url) != 'NA':
    search_url.append(get_next_url(init_url))
    init_url = get_next_url(init_url)

In [34]:
############### Create a document link list of a certain search result page
doc_link = list()
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')

for row in rows[1:]:
    cells = row.find_all('td')
    doc_link.append('https://www.sec.gov' + cells[1].a['href'])

In [35]:
############### Extract file identification info + HTML links
html_link = list()
filenum = list()
fd = list()
rp = list()
name = list()
cik = list()
sic = list()
filetype = list()
fye = list()
state = list()
bazip = list()
item8k = list()

for doc in doc_link:
    doc_resp = requests.get(doc)
    doc_str = doc_resp.text
# Save the HTML link
    soup = BeautifulSoup(doc_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile', summary='Document Format Files')
    rows = table_tag.find_all('tr')
    cells = rows[1].find_all('td')
    html_link.append('https://www.sec.gov' + cells[2].a['href'])
# Save the SEC filing number
    div_tag = soup.find('div', id='formHeader')
    secnum = div_tag.find('div', id='secNum')
    a = secnum.get_text().split()[3]
    filenum.append(a)
# Save the Filing Date and Reporting Period
    div_tag = soup.find('div', class_='formContent')
    dates = div_tag.find_all('div', class_='info')
    # Filing Date
    a = dates[0].get_text()
    fd.append(a)
    # Reporting Period
    b = dates[3].get_text()
    rp.append(b)
    # For 8K files, Save item info
    if type == '8-K':
        c = dates[4].get_text()
        clist = re.findall(r'\d.\d\d', c)
        c = ', '.join(clist)
        item8k.append(c)
    else :
        c = 'NA'
        item8k.append(c)
# Save the Company name and CIK
    div_tag = soup.find('div', class_='companyInfo')
    comname = div_tag.find('span', class_='companyName')
    # Company Name
    a = comname.get_text().split("\n")[0]
    name.append(a)
    # CIK
    b = comname.get_text().split("\n")[1]
    cik.append(b)
# Save Business Address ZIP 
    div_tag = soup.find_all('div', class_='mailer')[1]
    ba = div_tag.get_text()
    alist = re.findall(r'\d\d\d\d\d', ba)
    a = ', '.join(alist)
    bazip.append(a)
# Save SIC, File Type, Fiscal Year End and State of Incorporation
    div_tag = soup.find('div', class_='companyInfo')
    filinginfo = div_tag.find('p', class_='identInfo')
    # SIC
    a = filinginfo.get_text().split("|")[5].split("SIC")[1].split()[1]
    sic.append(a)
    # File Type
    b = filinginfo.get_text().split("|")[2].split("Type")[1].split(":")[1]
    filetype.append(b)
    # Fiscal Year End
    c = filinginfo.get_text().split("|")[2].split("Type")[0].split(":")[1]
    fye.append(c)
    # State
    d = filinginfo.get_text().split("|")[1].split(":")[1]
    state.append(d)
    
html_link = [w.replace('ix?doc=/', '') for w in html_link]
name = [w.replace(' (Filer)', '') for w in name]
cik = [w.replace('CIK: ', '') for w in cik]
cik = [w.replace(' (see all company filings)', '') for w in cik]

In [25]:
############### Download HTML into TXT files
for link in html_link:
    urllib.request.urlretrieve(link, '..\\filings\\raw\\'+str(filenum[html_link.index(link)])+'.txt')

In [36]:
############### Create Data Frame
d = {'cik': cik, 'name': name, 'fd': fd, 'rp': rp, 'filetype': filetype, 'item8k': item8k, 'filenum': filenum, 'bazip': bazip}
id_data = pd.DataFrame(data=d)
id_data

cik        name          fd          rp filetype  \
0    0000320193  Apple Inc.  2020-02-27  2020-02-26     8-K    
1    0000320193  Apple Inc.  2020-02-18  2020-02-17     8-K    
2    0000320193  Apple Inc.  2020-01-28  2020-01-28     8-K    
3    0000320193  Apple Inc.  2019-11-15  2019-11-07     8-K    
4    0000320193  Apple Inc.  2019-10-30  2019-10-30     8-K    
..          ...         ...         ...         ...      ...   
95   0000320193   APPLE INC  2011-04-20  2011-04-20     8-K    
96   0000320193   APPLE INC  2011-02-24  2011-02-23     8-K    
97   0000320193   APPLE INC  2011-01-18  2011-01-18     8-K    
98   0000320193   APPLE INC  2011-01-18  2011-01-17     8-K    
99   0000320193   APPLE INC  2010-11-17  2010-11-16     8-K    

              item8k               filenum  bazip  
0               5.07  0001193125-20-050884  95014  
1         7.01, 9.01  0001193125-20-039203  95014  
2         2.02, 9.01  0000320193-20-000008  95014  
3         8.01, 9.01  0001193125-19-292676  95014  
4         2.02, 9.01  0000320193-19-000117  95014  
..               ...                   ...    ...  
95        2.02, 9.01  0001193125-11-103732  95014  
96              5.07  0001193125-11-045411  95014  
97        2.02, 9.01  0001193125-11-009392  95014  
98              5.02  0001181431-11-003847  95014  
99  5.02, 5.03, 9.01  0001181431-10-056327  95014  

[100 rows x 8 columns]

In [ ]:
############ USE SEC_EDGAR_DOWNLOADER TO DOWNLOAD SEC FILES
#### BUT IT DOWNLOADS ***TXT FILES*** NOT ***HTML***
# from sec_edgar_downloader import Downloader

## Initialize a downloader instance.
## If no argument is passed to the constructor, the package
## will attempt to locate the user's downloads folder.
# dl = Downloader("filings/raw")

# Get all 10-Q filings for Apple
# dl.get("10-Q", "AAPL")